# Requirements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


❗️need to make sure python version is less than 3.12

In [2]:
!python --version

Python 3.11.13


In [3]:
!pip install transformers==4.31.0 peft==0.4.0 accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [4]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 28.5 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.18.2-py3-none-any.whl size=1763311 sha256=47fc124b9b483d17dbcf2c98d074d3262fd00485389bc5e0b29a95e972eb5947
  Stored in directory: /root/.cache/pip/wheels/80/2b/fe/f2625302f25976b9828a27d3d2567bdc1d587a1ff0ab42c8a9
Successfully built deepspeed


# Import and Setup

In [5]:
%cd drive/MyDrive/GenAI\ Project/VisCoT

/content/drive/MyDrive/GenAI Project/VisCoT


In [6]:
import os
import sys
import json
import torch
import transformers
from PIL import Image

from llava.train.train import train

print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch: 2.6.0+cu124
CUDA Available: True
GPU: NVIDIA A100-SXM4-40GB


# Download models

In [7]:
from huggingface_hub import snapshot_download

LLAVA_MODEL = "liuhaotian/llava-v1.5-7b"
VISION_TOWER = "openai/clip-vit-large-patch14-336"

print("Downloading models...\n")

print(f"1. {LLAVA_MODEL}")
llava_path = snapshot_download(repo_id=LLAVA_MODEL)
print(f"   Downloaded to: {llava_path}\n")

print(f"2. {VISION_TOWER}")
clip_path = snapshot_download(repo_id=VISION_TOWER)
print(f"   Downloaded to: {clip_path}\n")

print("Done!")


1. liuhaotian/llava-v1.5-7b


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

.gitattributes: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

   Downloaded to: /root/.cache/huggingface/hub/models--openai--clip-vit-large-patch14-336/snapshots/ce19dc912ca5cd21c8a653c79e251e808ccabcd1

Done!


# Dataset

In [8]:
!ls

assets		     llava			 recipeqa_gradcam_3.png  tools
checkpoints	     recipeqa_eval_results.json  results		 wandb
data		     recipeqa_gradcam_1.png	 scripts
gradcam_example.png  recipeqa_gradcam_2.png	 test.png


In [16]:
RECIPEQA_DIR = "../recipeqa"
TRAIN_JSON = os.path.join(RECIPEQA_DIR, "recipeqa_train.json")
TEST_JSON = os.path.join(RECIPEQA_DIR, "recipeqa_test.json")
TRAIN_IMAGE_FOLDER = os.path.join(RECIPEQA_DIR, "train")
TEST_IMAGE_FOLDER = os.path.join(RECIPEQA_DIR, "test")

# Check files
for name, path in [("Train JSON", TRAIN_JSON), ("TEST JSON", TEST_JSON)]:
    with open(path) as f:
        data = json.load(f)
    print(f"{name}: {len(data)} samples")

num_train_images = len([f for f in os.listdir(TRAIN_IMAGE_FOLDER) if f.endswith('.jpg')])
num_test_images = len([f for f in os.listdir(TEST_IMAGE_FOLDER) if f.endswith('.jpg')])
print(f"Train Images: {num_train_images}")
print(f"TEST Images: {num_test_images}")

# Show sample
with open(TRAIN_JSON) as f:
    print("\nExample training sample:")
    print(json.dumps(json.load(f)[0], indent=2))

Train JSON: 200 samples
TEST JSON: 80 samples
Train Images: 200
TEST Images: 80

Example training sample:
{
  "id": "recipeqa_1_0",
  "image": "1_0_false.jpg",
  "conversations": [
    {
      "from": "human",
      "value": "Identify if from left to right is the correct order of cooking this how to cook fava beans. Answer only 'true' or 'false'."
    },
    {
      "from": "gpt",
      "value": "false"
    }
  ]
}


In [17]:
# Ensure each human prompt explicitly includes an <image> token for multimodal training
import json, os
from llava.constants import DEFAULT_IMAGE_TOKEN

def add_image_token(path):
    with open(path) as f:
        data = json.load(f)
    updated = 0
    for sample in data:
        conv = sample.get("conversations", [])
        if conv and conv[0].get("from") == "human":
            text = conv[0].get("value", "")
            if DEFAULT_IMAGE_TOKEN not in text:
                conv[0]["value"] = f"{DEFAULT_IMAGE_TOKEN}\n{text}"
                updated += 1
    with open(path, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"{os.path.basename(path)}: added <image> to {updated} samples")

for path in [TRAIN_JSON, TEST_JSON]:
    add_image_token(path)

recipeqa_train.json: added <image> to 200 samples
recipeqa_test.json: added <image> to 80 samples


# Configs

In [18]:
OUTPUT_DIR = "./checkpoints/viscot_recipe_prefix"

### experiment 1
NUM_EPOCHS = 2
BATCH_SIZE = 2
GRAD_ACCUM = 8
# LR = 1e-3

### experiment 2
# NUM_EPOCHS = 3
LR = 2e-4

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Output: {OUTPUT_DIR}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch Size: {BATCH_SIZE} x {GRAD_ACCUM} = {BATCH_SIZE*GRAD_ACCUM}")
print(f"Learning Rate: {LR}")

Output: ./checkpoints/viscot_recipe_prefix
Epochs: 2
Batch Size: 2 x 8 = 16
Learning Rate: 0.0002


In [20]:
import warnings
warnings.filterwarnings('ignore')

# Set arguments
sys.argv = [
    'train',
    '--model_name_or_path', LLAVA_MODEL,
    '--version', 'v1',
    '--vision_tower', VISION_TOWER,
    '--mm_projector_type', 'mlp2x_gelu',
    '--mm_vision_select_layer', '-2',
    '--mm_use_im_start_end', 'False',
    '--mm_use_im_patch_token', 'False',

    # Prefix tuning: freeze backbone, train projector only
    '--freeze_backbone', 'True',
    '--tune_mm_mlp_adapter', 'True',
    '--freeze_mm_mlp_adapter', 'False',

    # Data
    '--data_path', TRAIN_JSON,
    '--is_multimodal', 'True',
    '--image_folder', TRAIN_IMAGE_FOLDER,
    '--image_aspect_ratio', 'pad',
    '--group_by_modality_length', 'False',
    '--lazy_preprocess', 'True',

    # Training parameters (use variables from cell-15)
    '--bf16', 'False',
    '--fp16', 'True',
    '--output_dir', OUTPUT_DIR,
    '--num_train_epochs', str(NUM_EPOCHS),
    '--per_device_train_batch_size', str(BATCH_SIZE),
    '--gradient_accumulation_steps', str(GRAD_ACCUM),
    '--evaluation_strategy', 'no',
    '--save_strategy', 'epoch',
    '--save_total_limit', '3',
    '--learning_rate', str(LR),
    '--weight_decay', '0.0',
    '--warmup_ratio', '0.03',
    '--lr_scheduler_type', 'cosine',
    '--logging_steps', '10',
    '--tf32', 'False',
    '--model_max_length', '2048',
    '--gradient_checkpointing', 'True',
    '--dataloader_num_workers', '4',
    # '--report_to', 'wandb',
    '--report_to', 'tensorboard',
]

print("Starting training...")
print("Method: Prefix tuning (train projector only, freeze backbone)\n")
print(f"Effective batch size: {BATCH_SIZE} x {GRAD_ACCUM} = {BATCH_SIZE*GRAD_ACCUM}")
print(f"Total epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LR}\n")

try:
    train()
    print("\nTraining completed!")
except Exception as e:
    print(f"\nError: {e}")
    import traceback
    traceback.print_exc()

You are using a model of type llava to instantiate a model of type viscot_llava. This is not supported for all configurations of models and can yield errors.


Starting training...
Method: Prefix tuning (train projector only, freeze backbone)

Effective batch size: 2 x 8 = 16
Total epochs: 2
Learning rate: 0.0002



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlavaLlamaForCausalLM were not initialized from the model checkpoint at liuhaotian/llava-v1.5-7b and are newly initialized: ['model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.8.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.18.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.13.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.pre_layrnorm.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.19.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.15.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.3.self_attn.k_proj.weight', 'model.v

Formatting inputs...Skip in lazy mode


Step,Training Loss
10,1.856900
20,0.367300



Training completed!


# Checking

# Evaluate

In [21]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
import torch

checkpoints = sorted([d for d in os.listdir(OUTPUT_DIR) if d.startswith('checkpoint-')])

if checkpoints:
    checkpoint_path = os.path.join(OUTPUT_DIR, checkpoints[-1])
    print(f"Loading checkpoint: {checkpoint_path}")

    print("Loading base LLaVA model...")
    print(f"Model path: {LLAVA_MODEL}")
    print(f"Vision tower: {VISION_TOWER}")

    # Load base model
    tokenizer, model, image_processor, context_len = load_pretrained_model(
        model_path=LLAVA_MODEL,
        model_base=None,
        model_name=get_model_name_from_path(LLAVA_MODEL),
        load_8bit=False,
        load_4bit=False,
        # torch_dtype=torch.float16,
        # low_cpu_mem_usage=False,
        device_map={"": "cuda:0"},
    )
    model = model.cuda().half()


    print("\nInitializing vision tower...")
    vision_tower = model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model()

    print("Moving model to GPU and converting to FP16...")
    model = model.cuda()

    # Convert ALL components to FP16
    model = model.half()  # Convert entire model to FP16

    print("✓ Converted to FP16 successfully")

    # Test vision tower
    print("\nTesting vision tower...")
    dummy_img = torch.randn(1, 3, 336, 336).half().cuda()
    with torch.no_grad():
        test_features = vision_tower(dummy_img)

    print(f"Vision output shape: {test_features.shape}")
    print(f"Has NaN: {torch.isnan(test_features).any().item()}")
    print(f"Has Inf: {torch.isinf(test_features).any().item()}")

    if torch.isnan(test_features).any():
        print("\n ERROR: Vision tower still produces NaN!")
        print("Trying to reload vision tower from scratch...")
        from transformers import CLIPVisionModel
        vision_model = CLIPVisionModel.from_pretrained(VISION_TOWER).cuda().half()
        model.get_vision_tower().vision_tower = vision_model

        # Test again
        with torch.no_grad():
            test_features = vision_tower(dummy_img)
        print(f"After reload - Has NaN: {torch.isnan(test_features).any().item()}")

    # Load fine-tuned projector weights
    mm_projector_path = os.path.join(checkpoint_path, 'mm_projector.bin')
    print(f"\nLoading fine-tuned projector from: {mm_projector_path}")

    projector_weights = torch.load(mm_projector_path, map_location='cpu')

    # Convert projector weights to FP16 and move to GPU
    for key in projector_weights.keys():
        projector_weights[key] = projector_weights[key].half().cuda()

    # Load the weights
    msg = model.load_state_dict(projector_weights, strict=False)
    print(f"Loaded projector - Missing: {len(msg.missing_keys)}, Unexpected: {len(msg.unexpected_keys)}")

    # Final sanity check
    print("\n" + "="*60)
    print("Final model state:")
    print(f"  Device: {next(model.parameters()).device}")
    print(f"  Dtype: {next(model.parameters()).dtype}")

    # Test full pipeline
    print("\nFinal vision tower test:")
    with torch.no_grad():
        vision_out = vision_tower(dummy_img)
        projector_out = model.get_model().mm_projector(vision_out)

    print(f"  Vision output has NaN: {torch.isnan(vision_out).any().item()}")
    print(f"  Projector output has NaN: {torch.isnan(projector_out).any().item()}")
    print(f"  Vision output mean: {vision_out.mean().item()}")

    if not torch.isnan(projector_out).any():
        print("\n✓ Model loaded successfully without NaN!")
    else:
        print("\n❌ WARNING: Still have NaN in the pipeline!")
    print("="*60)

else:
    print("No checkpoints found!")

You are using a model of type llava to instantiate a model of type viscot_llava. This is not supported for all configurations of models and can yield errors.


Loading checkpoint: ./checkpoints/viscot_recipe_prefix/checkpoint-24
Loading base LLaVA model...
Model path: liuhaotian/llava-v1.5-7b
Vision tower: openai/clip-vit-large-patch14-336


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlavaLlamaForCausalLM were not initialized from the model checkpoint at liuhaotian/llava-v1.5-7b and are newly initialized: ['model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.8.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.18.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.13.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.pre_layrnorm.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.19.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.15.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.3.self_attn.k_proj.weight', 'model.v


Initializing vision tower...
Moving model to GPU and converting to FP16...
✓ Converted to FP16 successfully

Testing vision tower...
Vision output shape: torch.Size([1, 576, 1024])
Has NaN: True
Has Inf: False

 ERROR: Vision tower still produces NaN!
Trying to reload vision tower from scratch...
After reload - Has NaN: False

Loading fine-tuned projector from: ./checkpoints/viscot_recipe_prefix/checkpoint-24/mm_projector.bin
Loaded projector - Missing: 714, Unexpected: 0

Final model state:
  Device: cuda:0
  Dtype: torch.float16

Final vision tower test:
  Vision output has NaN: False
  Projector output has NaN: False
  Vision output mean: 0.05450439453125

✓ Model loaded successfully without NaN!


In [24]:
import os
import json
from PIL import Image
import torch
from llava.mm_utils import tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN


def _safe_div(num: float, den: float) -> float:
    return num / den if den else 0.0


def evaluate(model, tokenizer, image_processor, test_json, image_folder):
    """Evaluate model on a true/false task with full precision/recall/F1 metrics."""
    with open(test_json, 'r') as f:
        data = json.load(f)

    vocab_size = len(tokenizer)

    results = []
    unknown = 0
    model.eval()

    for i, sample in enumerate(data):
        img_path = os.path.join(image_folder, sample['image'])
        image = Image.open(img_path).convert('RGB')

        # Ensure the prompt starts with an <image> token
        prompt = sample['conversations'][0]['value']
        if DEFAULT_IMAGE_TOKEN not in prompt:
            prompt = f"{DEFAULT_IMAGE_TOKEN}{prompt}"

        ground_truth = sample['conversations'][1]['value'].strip().lower()
        # Expected labels: "true" or "false"

        # Process image
        img_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()

        # Tokenize prompt
        input_ids = tokenizer_image_token(
            prompt,
            tokenizer,
            IMAGE_TOKEN_INDEX,
            return_tensors='pt'
        ).unsqueeze(0).cuda()

        # Generate model output
        with torch.no_grad():
            out = model.generate(
                input_ids,
                images=img_tensor,
                do_sample=False,
                max_new_tokens=128,
                use_cache=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
            )

        # Decode output
        generated_ids = out[0, input_ids.shape[1]:].tolist()

        valid_ids = []
        for token_id in generated_ids:
            if token_id == IMAGE_TOKEN_INDEX or token_id < 0 or token_id >= vocab_size:
                continue
            if token_id == tokenizer.eos_token_id:
                break
            valid_ids.append(token_id)

        output = tokenizer.decode(valid_ids, skip_special_tokens=True).strip()
        output_lower = output.lower()

        # ===== True/False Classification =====
        if "true" in output_lower:
            predicted = "true"
        elif "false" in output_lower:
            predicted = "false"
        else:
            predicted = "unknown"

        is_correct = predicted == ground_truth

        results.append({
            'id': sample['id'],
            'image': sample['image'],
            'ground_truth': ground_truth,
            'predicted': predicted,
            'full_output': output,
            'correct': is_correct
        })

        if predicted == 'unknown':
            unknown += 1

        # --- Print progress ---
        print(f"[{i+1}/{len(data)}] {sample['id']}")
        print(f"  GT: {ground_truth}")
        print(f"  Pred: {predicted}")
        print(f"  Full output: {output[:200]}")
        print("  ✓ Correct" if is_correct else "  ✗ Wrong")

    # ======================================================
    # Compute metrics
    # ======================================================

    labels = ["true", "false"]
    conf = {c: {"tp": 0, "fp": 0, "fn": 0, "count": 0} for c in labels}

    for r in results:
        gt = r["ground_truth"]
        pred = r["predicted"]

        if gt in labels:
            conf[gt]["count"] += 1

        if pred not in labels:
            if gt in labels:
                conf[gt]["fn"] += 1
            continue

        for c in labels:
            if pred == c and gt == c:
                conf[c]["tp"] += 1
            elif pred == c and gt != c:
                conf[c]["fp"] += 1
            elif gt == c and pred != c:
                conf[c]["fn"] += 1

    metrics = {}
    for c in labels:
        tp, fp, fn = conf[c]["tp"], conf[c]["fp"], conf[c]["fn"]
        prec = _safe_div(tp, tp + fp)
        rec = _safe_div(tp, tp + fn)
        f1 = _safe_div(2 * prec * rec, prec + rec) if (prec + rec) else 0.0

        metrics[f"{c}_precision"] = prec
        metrics[f"{c}_recall"] = rec
        metrics[f"{c}_f1"] = f1

    # Macro average
    metrics["macro_precision"] = (metrics["true_precision"] + metrics["false_precision"]) / 2
    metrics["macro_recall"] = (metrics["true_recall"] + metrics["false_recall"]) / 2
    metrics["macro_f1"] = (metrics["true_f1"] + metrics["false_f1"]) / 2

    total_count = conf["true"]["count"] + conf["false"]["count"]

    # Weighted metrics
    metrics["weighted_precision"] = (
        (conf["true"]["count"] * metrics["true_precision"] +
         conf["false"]["count"] * metrics["false_precision"]) / total_count
    )

    metrics["weighted_recall"] = (
        (conf["true"]["count"] * metrics["true_recall"] +
         conf["false"]["count"] * metrics["false_recall"]) / total_count
    )

    metrics["weighted_f1"] = (
        (conf["true"]["count"] * metrics["true_f1"] +
         conf["false"]["count"] * metrics["false_f1"]) / total_count
    )

    # Accuracy and unknown ratio
    metrics["accuracy"] = sum(1 for r in results if r["correct"]) / len(results)
    metrics["unknown_frac"] = _safe_div(unknown, len(results))

    # ======================================================
    # Print Summary
    # ======================================================
    print("\n" + "=" * 60)
    print(
        f"Accuracy: {metrics['accuracy']:.4f} | "
        f"Weighted P/R/F1: {metrics['weighted_precision']:.4f} / "
        f"{metrics['weighted_recall']:.4f} / "
        f"{metrics['weighted_f1']:.4f} | "
        f"Unknown: {metrics['unknown_frac']:.4f}"
    )

    for c in labels:
        print(
            f"{c.capitalize():>6} - "
            f"P: {metrics[f'{c}_precision']:.4f} "
            f"R: {metrics[f'{c}_recall']:.4f} "
            f"F1: {metrics[f'{c}_f1']:.4f} "
            f"(count={conf[c]['count']})"
        )
    print("=" * 60)

    return results, metrics

In [25]:

# Run evaluation
results, metrics = evaluate(model, tokenizer, image_processor, TEST_JSON, TEST_IMAGE_FOLDER)


[1/80] recipeqa_51_0
  GT: false
  Pred: false
  Full output: false
  ✓ Correct
[2/80] recipeqa_51_1
  GT: false
  Pred: false
  Full output: false
  ✓ Correct
[3/80] recipeqa_51_2
  GT: true
  Pred: false
  Full output: false
  ✗ Wrong
[4/80] recipeqa_51_3
  GT: false
  Pred: false
  Full output: false
  ✓ Correct
[5/80] recipeqa_52_0
  GT: false
  Pred: unknown
  Full output: 
  ✗ Wrong
[6/80] recipeqa_52_1
  GT: false
  Pred: false
  Full output: false
  ✓ Correct
[7/80] recipeqa_52_2
  GT: false
  Pred: false
  Full output: false
  ✓ Correct
[8/80] recipeqa_52_3
  GT: true
  Pred: unknown
  Full output: 
  ✗ Wrong
[9/80] recipeqa_53_0
  GT: false
  Pred: unknown
  Full output: 
  ✗ Wrong
[10/80] recipeqa_53_1
  GT: false
  Pred: false
  Full output: false
  ✓ Correct
[11/80] recipeqa_53_2
  GT: false
  Pred: unknown
  Full output: 
  ✗ Wrong
[12/80] recipeqa_53_3
  GT: true
  Pred: false
  Full output: false
  ✗ Wrong
[13/80] recipeqa_54_0
  GT: false
  Pred: unknown
  Full output: